In [ ]:
import os
import pandas as pd
from PIL import Image
import numpy as np

run = True

In [ ]:
def generate_mil_csv_file(images_path, masks_path, csv_path, meta_data, image_postfix=".jpg", mask_postfix=".bmp", split_percentage=90, new_mask_path=None, mask_crop=False, drop=False):
    
    # have to call for each dataset that contains images + masks
    # finding image (.image_postfix) and mask (.mask_postfix) pairs for same base filename
    # this works if your_img_dir/A0001.jpg and your_mask_dir/A0001.bmp
    # optic nerve head
    # converts all the masks - 0 background, 255 foreground if new_mask_path is not None
    
    mode = []
    
    import glob
    # need one file for each concept 
    image_path = glob.glob(images_path) # list(set(os.listdir(images_path)) - {'desktop.ini'})
    
    
    image_id = [os.path.basename(x).replace(image_postfix, "") for x in image_path]
    
    
    
    print(image_id[0:5])
    
    image_path = [os.path.join(images_path, x) for x in image_path]
    a = {"image_path" : image_path, "id" : image_id}
    a = pd.DataFrame(data={"img_path" : image_path}, index=image_id)
    
    has_roi = []
    
    if masks_path is not None:
        mask_path  = glob.glob(masks_path)
        mask_id = [os.path.basename(x).replace(mask_postfix, "").replace("mask_", "").replace("DiskMask", "") for x in mask_path]
        mask_path = [os.path.join(masks_path, x) for x in mask_path]
                
        # b = {"msk_path" : msk_path, "id" : mask_id, "has_roi" : has_roi}
        b = pd.DataFrame(data={"msk_path" : mask_path}, index=mask_id)
        
        print(a.head())
        print(b.head())
        
        csv_data = pd.concat([a, b], axis=1, join="outer")
        
        csv_data["has_mask"] = [True] * len(a.index) # this has a bug
        
        
    else: # if no mask available
        csv_data = a
        csv_data["msk_path"] = [None] * len(a.index)
        csv_data["has_mask"] = [False] * len(a.index)
        
    for key, value in meta_data.items():
        csv_data[key] = [value] * len(a.index)  
        
    
    csv_data = csv_data.sample(frac=1, random_state=19)

    percent_train = int(len(csv_data)/100*split_percentage)
    percent_val = len(csv_data)-percent_train
    
    mode.extend(["train" for i in range(percent_train)])
    mode.extend(["val" for i in range(percent_val)])
    
    csv_data["mode"] = mode

    print(csv_data)
    
    if drop:
        csv_data = csv_data.dropna(subset=['msk_path'])
    
    csv_data.to_csv(csv_path, sep=';')
            

In [ ]:
if run:
    images_path = r"E:/Christina/MIL_Data/octa_whole_cirrus_6_onh_glaucoma/images/*.bmp"
    masks_path  = None
    csv_path    = r"mil_data_cirrus_glaucoma.csv"
    meta_data = { "glaucoma":True, "dr":None, "healthy":False, "ONH":True, "FAZ":False}
    generate_mil_csv_file(images_path, masks_path, csv_path, meta_data, image_postfix=".bmp", mask_postfix=None, split_percentage=80, new_mask_path=None, drop=False)

In [ ]:
if run:
    images_path = r"E:/Christina/MIL_Data/octa_whole_cirrus_6_onh_healthy/images/*.bmp"
    masks_path  = None
    csv_path    = r"mil_data_cirrus_healthy.csv"
    meta_data = { "glaucoma":False, "dr":False, "healthy":True, "ONH":True, "FAZ":False}
    generate_mil_csv_file(images_path, masks_path, csv_path, meta_data, image_postfix=".bmp", mask_postfix=None, split_percentage=80, new_mask_path=None, drop=False)

In [ ]:
if run:
    images_path = r"E:/Christina/MIL_Data/octa_whole_octa500_6_faz_unknown/images/*.bmp"
    masks_path  = None
    csv_path    = r"mil_data_octa500_unknown.csv"
    meta_data = { "glaucoma":None, "dr":None, "healthy":None, "ONH":False, "FAZ":True}
    generate_mil_csv_file(images_path, masks_path, csv_path, meta_data, image_postfix=".bmp", mask_postfix=None, split_percentage=80, new_mask_path=None, drop=False)